**Prod Version of DevEnv**

**This notebook is run using nbconvert after devenv service starts**

In [1]:
!pip install twitter
!pip install feedparser
!pip install schedule
!pip install kafka-python
!pip install redis

In [2]:
from time import sleep
from json import dumps, loads, load
from kafka import KafkaProducer
from datetime import datetime, timedelta
from feedparser import parse
from urllib.request import urlopen
from twitter import Twitter
import schedule

In [3]:
tickers = ["MSFT","AAPL"]

producer = KafkaProducer(
    bootstrap_servers=['broker:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

In [4]:
def DownloadStreams():
    f_data = {}
    for ticker in tickers:
        try:
            y_finance = load(urlopen("https://query1.finance.yahoo.com/v7/finance/chart/"+ticker+"?range=1h&interval=1h"))
            meta = y_finance['chart']['result'][0]['meta']
            timestamp = meta['regularMarketTime']
            price = meta['regularMarketPrice']
            f_data[ticker] = {"Ticker": ticker, "Timestamp":timestamp, "Price":price}
        except:
            print("Exception in Download Stream for "+ticker)
    return f_data

def UploadStreamsToKafka(f_data):
    for key in f_data.keys():
        producer.send('finance_ticker', value=f_data[key])
    producer.flush()

def HourlyJob():
    # Check, Download & Parse each stream
    f_data = DownloadStreams()
    # Upload cleaned events into Kafka Topics
    UploadStreamsToKafka(f_data)
    print("Hourly Run Events Complete - Sleeping now")

In [5]:
schedule.clear()
HourlyJob()

schedule.every(1).hour.do(HourlyJob)
starttime = datetime.now()
endtime = starttime + timedelta(minutes=5)

while True: #datetime.now()<endtime:
    schedule.run_pending()
    sleep(100)
    
schedule.clear()

Hourly Run Events Complete - Sleeping now
Hourly Run Events Complete - Sleeping now
Hourly Run Events Complete - Sleeping now


KeyboardInterrupt: 

In [ ]:
HourlyJob()